In [ ]:
from IPython.core.display import HTML, display
display(HTML('<script>Jupyter.notebook.clear_all_output(); '+
             'Jupyter.notebook.kernel.restart();</script>'))

In [1]:

import numpy as np;
import plotly.graph_objects as go
from numpy.linalg import *
from scipy.signal import periodogram, convolve, remez, freqz, find_peaks, lfilter, welch;
from scipy.signal.windows import chebwin;
import pandas as pds;
from datetime import datetime
from linregs import *;
from plots import *;
from msk import *;
# from dbutils import *;
from equalization import *; 
from jack_record import record;
import os
from datetime import datetime
import subprocess
    

import plotly.io as pio
import profile; 


In [6]:
### File options ###
home="/media/jradema/ddced86b-d0c5-4812-8de0-2cadf793097c/home/prwang"

dist=60
# tx_amp = 0.08 # 60Vrms
# tx_amp = 0.06 # 40Vrms
tx_amp = 0.015 # 10Vrms
# tx_amp = 0.025 # 20Vrms
volt = 40 #np.round(tx_amp*1000).astype("uint8")
# trial=1
num_trials = 3
trial_start = 0
angles = range(-90,91,5)
nch=7

msk_ord = 0
data_rate = 500
fs = 2e5
nbits = 10000

rx_dir=os.path.join(home,"rx_outputs");
tx_dir=os.path.join(home,"tx_outputs");
expname='River_PAB2_Van_Atta';
now = datetime.now()
datestr=now.strftime("%m-%d-%Y");
datestr="02-13-2023"
expset=expname + "_" + datestr;

try:
    os.mkdir(os.path.join(rx_dir,expset));
    print(f"Created directory {expset} in {rx_dir} for experiment set.")
except FileExistsError as error:
    print(f"Directory {expset} already exists")
    
rx_root = f"fixed_vanatta4x2_stag_dr={data_rate}bps_ord={msk_ord}_Vrms={volt}_poly_011A_tx_{dist}m_1m_single_foam_sep_purui_rx";

#rx_filename = f"vanatta4x2_{nch}ch_rx_nostag_006B_006F_006A_006C_x_001A_004A_004B_004D_18,5kfc_0,0deg_0,5kbps_usrp_2,5m_depth_005B_{volt}Vrms_{dist}m_{dist}m_1m_2foam_sep_{trial}.dat";

tx_filename = f"msk_data_dr={data_rate}_ord={msk_ord}_nbits=10k_fs=2e5.dat";
tx_path = os.path.join(tx_dir,tx_filename);

Directory River_PAB2_Van_Atta_02-13-2023 already exists


In [7]:
def get_wf( filename, spb, Fc_c, data_rate, msk_ord, Fs, stretch=1):
    Fc_t = -data_rate * (msk_ord*2+3) /4;
    Fs_new = data_rate * spb;
    segment=np.fromfile(filename,dtype=np.float32).reshape([-1, 7])[:, :]
#     pcs(segment[:,0])
#     print(f'segment has shape {segment.shape}, Fs={Fs}, Fs_new={Fs_new}')
    return mixer_cfo(Fc_c, [Fc_t, -Fc_t], stretch, Fs, segment, Fs_new)

def fm0_bpsk1(source):
    last_sym = 1;
    polarity = -1;
    output = np.zeros((source.size * 2));
    for i in range(source.size):
        if last_sym != source[i]: # xor = 1;
            output[i*2:i*2+2] = -polarity, -polarity
            polarity = -polarity;
        else: # xor = 0
            output[i*2:i*2+2] = -polarity, polarity
        last_sym = source[i];
    return output;

In [8]:
### EQUALIZE vs. VARIABLE (angle, range, etc.) ### 

# Nfft = 2**(np.ceil(np.log2(Fs*10)));
# att = 300; # attenuation of chebwin in dB

# var_list = [22,23,24,25,27,28,29,30,32,34,36,38,40,42,44,47]
# var_list = [40,47,50,53,56,60,63,66,70,78,80,81,83]
# var_list = [81,83,90,100]
# var_list = [10,20,30,40,50,60]
spb=2
source = pn15_seq[:nbits] * 2 - 1;    

fw_len=15
bw_len=4
train_len = 1000;
lam = 0.999
Fc_c = 18500
Fs=192000

max_step = source.size-train_len

preamble_bits = source[:train_len];
exp_bits = source[train_len:]

template_fm0 = fm0_bpsk1(preamble_bits);
template = fm0_subcar_filt(template_fm0, spb)

var_list = angles
snr_arr = np.zeros((len(var_list),num_trials))
ber_arr = np.zeros((len(var_list),num_trials))

# spectrums = np.zeros((len(var_list),Nfft.astype("uint32")))

for i in range(0,len(var_list)):
    snrs = []
    bers = []
    
    ang = var_list[i]
    print(f"Angle: {ang}")
    
   
    for trial in range(trial_start,trial_start+num_trials):
        rx_filename = f"{rx_root}_ang={ang}deg_{trial}.dat"
        rx_path = os.path.join(rx_dir,expset,rx_filename); 

        if not os.path.isfile(rx_path):
            print(f'Could not find trial {trial}, skipping...')
            continue

        print(f"Trial: {trial}")

        dat1 = get_wf(rx_path, spb, Fc_c, data_rate, msk_ord, Fs,)

        # sig = passband[:, 6];
        # window_size = np.floor(len(sig)/100);
        # window = chebwin(window_size.astype("uint32"), att);

        # spectrums[i,:] = welch(sig,Fs,window=window,nfft=Nfft);

        # print(f'dat has shape{dat1.shape}')
        rxfilt = fm0_lowpass_filt(dat1, spb);
        # print(f"rxfilt has shape {rxfilt.shape}")
        scale,corr = linreg_mc_od(rxfilt, template);
        offset = fw_len//3
        #offset  =0;

        pos= np.argmax(corr[:]) + (offset - preamble_bits.size) * spb + 1;
        print(f'pos={pos}')
        bpsk_sample = fm0_bb_decimate(rxfilt[pos:, :], spb, True, True);

        try:
            d_all, e_all, eq = dfe_sfo_main(bpsk_sample, max_step, spb, lam, fw_len, bw_len, preamble_bits, LDLT_RLS);

            ber = np.sum(source[train_len:train_len+d_all.size] != d_all) / max_step;
            bplot = np.cumsum(source[train_len:train_len+d_all.size] != d_all) ;
            avg_len = 20
            mse = 10*np.log10(convolve(np.abs(e_all)**2,np.ones((avg_len,)))/avg_len)
            snr = d_all.size/np.sum(np.abs(e_all+d_all-exp_bits)**2);

        except:
            snr = 1
            ber = 0.5

        print('snr is %.2f dB' % (10*np.log10(snr)))
        print('ber is %.2E' % ber)
        
        if ber < 1e-4:
            ber = 1e-4
        
        snr_arr[i,trial] = 10*np.log10(snr)
        ber_arr[i,trial] = ber

#     snr_arr[i] = 10*np.log10(np.median(snrs));
#     ber_arr[i] = np.median(bers) if np.median(bers) >= 1e-4 else 1e-4;

# pcs(ber_arr)
# pcs(snr_arr)
    

Angle: -90
Trial: 0
[ -267.72766-1113.6086j    1547.695    -77.089325j
 -1088.551   -405.74017j  -2209.8977 +2200.324j
  -190.7937  -942.014j    -2357.9307 +2060.3772j
  1639.7316  +834.8818j    1211.5715 -2723.1658j
   699.3678 +1316.4568j     469.18994 +777.28906j
   108.26555 -865.13306j   1294.3856 -2060.8306j
 -1214.4526 +1728.2278j    2258.3718  +745.848j   ]
pos=744
snr is 5.55 dB
ber is 1.27E-02
Trial: 1
[  147.18417 +1280.6141j    -64.80267 -1549.12j
    -9.539825 -525.80115j   518.2007  +1194.7378j
 -1669.7386  +1470.92j      316.1031  -1109.7522j
 -1294.5273  -2840.4229j    359.48343 +3069.472j
  1247.2334   +499.58514j -2372.8818  -1663.835j
   585.4616  +2227.6997j    568.4227  +1936.171j
  1506.7692  -1617.7694j    782.40405 -1101.8091j ]
pos=732
snr is 5.30 dB
ber is 1.74E-02
Trial: 2
[ -476.11588 +2.2370442e+03j    90.359406+9.6017792e+02j
  -528.22577 -5.4186731e+02j  1145.8071  -1.7225514e+03j
  -244.53238 +1.3729208e+02j  -676.7533  +7.5775488e+02j
 -2148.9583  +5.65

pos=723
snr is 8.03 dB
ber is 1.33E-03
Angle: -55
Trial: 0
[    7.5879517-1475.8798j  -1803.5696   -2973.7288j
  -434.60764  +2384.872j   -1881.6498   -4321.0205j
 -2809.0366   -1815.0945j  -2907.363    -2051.976j
   206.0346    -637.0166j   2093.583    +1003.75j
  -920.1941    +669.8278j   3629.875     -629.1572j
  1906.7993   +2156.9565j   -326.85788  -5750.8174j
  -569.5828   +1088.1985j   1812.9587    -327.81714j]
pos=754
snr is 8.67 dB
ber is 3.33E-04
Trial: 1
[  844.1249  +723.11255j  -985.71204 -314.30838j  -175.19507+1851.5443j
   989.4095 +2117.747j   -2486.9087  +675.70544j   397.7495  +992.3661j
   712.89404 -830.4225j  -2340.5923 +1848.6826j   1882.9863 +1148.3458j
 -1034.5358 -2314.1821j   2043.81    +541.86487j -1006.09296  +75.9613j
  -497.6062 +1282.7039j   -218.85658 -868.04376j]
pos=733
snr is 8.48 dB
ber is 1.11E-03
Trial: 2
[-1671.7329  +154.84204j  -692.0279 +1165.2926j   1732.9806  +139.96661j
   -97.90117-1038.155j     592.0795  -202.35934j  -446.25433 -882.0775j

pos=735
snr is 9.63 dB
ber is 5.56E-04
Angle: -20
Trial: 0
[-2234.38    -1250.8289j   -1585.9355   +998.56946j
   228.4259     +9.893997j   247.83966  +934.4356j
  1243.1042  -1818.6277j     250.0146  +1034.1653j
  1714.7034  -2051.3916j   -2038.9364   -257.2485j
 -2542.6025  +1088.3809j      -8.956116  +61.248306j
 -1474.1471   +856.01685j   -699.95404  -842.90936j
   954.8165   -394.27066j   -219.10162  +796.3099j  ]
pos=740
snr is 8.78 dB
ber is 7.78E-04
Trial: 1
[1534.0049   +91.02185j   426.8495  -961.81195j   658.7263  +878.28796j
  179.83627  +94.57236j  1091.3043  -363.28247j   -65.50055+1899.6951j
 2343.5974  +942.05005j   783.2934   +29.730099j -149.94818 +871.69543j
 2243.0774  -122.74278j  2476.8977 -1237.083j     622.71924-1459.6346j
 -343.26978 +804.2263j    954.6441  -158.33072j ]
pos=738
snr is 8.87 dB
ber is 8.89E-04
Trial: 2
[ 1717.4196  -1580.3998j     14.214058  +68.27261j
  -546.8999   +999.5113j    299.19904  +743.54395j
  1200.6866   +863.1993j   1070.8546  -1903

pos=724
snr is 10.11 dB
ber is 2.22E-04
Angle: 15
Trial: 0
[  -17.887207+1123.2961j    2227.566   -1746.5314j
   464.4248  -2068.615j    -1073.4325  +1292.8589j
   958.7379   +107.737976j -1886.7397  +1140.425j
 -1283.5009   -831.94635j   1162.0874  +1480.823j
  -591.9452   +166.6998j     972.3886  -2769.4675j
  -611.4684   +594.81024j  -1385.3542  +1396.863j
   191.82663  +121.91379j   -543.08936  +260.06174j ]
pos=710
snr is 8.67 dB
ber is 6.67E-04
Trial: 1
[-1433.221  +1.2692563e+03j  3161.8445 +2.0161936e+03j
 -1177.5546 -3.7098669e+02j  -120.74811-1.6462463e+03j
  -277.6188 +1.3872660e+03j   -89.04614-1.2362900e+03j
  -427.1234 -2.5079193e+00j  2732.0803 +1.0474210e+03j
  -189.59958-3.6727820e+02j -3154.1023 +3.9399805e+03j
  2480.4255 -7.2101477e+02j -1545.6646 -8.0701086e+02j
    94.03821-1.4739749e+03j  -657.3546 +2.6431047e+03j]
pos=737
snr is 9.00 dB
ber is 2.00E-03
Trial: 2
[-1523.2733  +543.97705j  -861.3821  -983.9496j    590.8267  -557.70996j
  1657.928   -928.762j   -189

pos=739
snr is 9.40 dB
ber is 8.89E-04
Angle: 50
Trial: 0
[ 1440.865   -385.67816j  -1774.4939 -2445.4814j
  -363.56842-1346.2709j      47.48688 +531.81323j
  -667.43726+2409.7046j     812.35876 -605.0435j
  -654.78314-1544.9722j    2365.0503  +391.15997j
  -501.22546-1283.5701j   -3195.2825 -2011.7777j
 -2361.1323 -3624.9783j    2501.326   +107.218376j
  -531.77466 +364.6072j   -1317.039     -9.031738j]
pos=730
snr is 9.73 dB
ber is 2.22E-04
Trial: 1
[ -730.5889 +5.7765509e+02j  -552.5839 -5.5845001e+02j
   450.06808-3.1580762e+02j   113.82501-1.3997083e+03j
  -460.58795+5.3176343e+02j  -932.92615+9.6197510e-01j
 -2184.1267 +1.4548544e+02j -1359.4469 -4.9741685e+03j
  2530.2422 -3.4813440e+02j    50.1362 -1.7727432e+02j
  2403.1245 -3.3026819e+02j -2212.7727 +5.8640430e+02j
   736.75934-2.6265863e+02j  1306.5803 +1.0139856e+03j]
pos=747
snr is 10.09 dB
ber is 1.11E-04
Trial: 2
[ 1399.8722  -86.06767j  -858.4004 +603.54126j -1546.2231 +605.47314j
 -1549.4421-2761.8962j  -1766.8446 -502

pos=750
snr is 4.10 dB
ber is 3.93E-02
Angle: 85
Trial: 0
[  113.71048-1415.9973j    -147.47461 +295.87695j
  2356.3862    -3.257843j  -614.7694 -1332.3059j
  1059.3344  +916.9503j    1155.298   +718.9358j
 -1292.7872 +1913.0326j    2589.9695 +1947.0604j
  2352.6284  -195.17618j    309.1215 -2035.3203j
   724.4632 -1003.j       -2659.608  +1230.805j
  1317.4906  -418.77124j    178.48941-1838.0519j  ]
pos=737
snr is 5.16 dB
ber is 1.99E-02
Trial: 1
[ -158.3623   +1177.5762j  -1046.6161    -843.3944j
   131.80148   -484.86078j  1609.1511    -251.22253j
 -1086.1538   -1412.1632j  -1797.2146    +495.76837j
  1240.3953   -2174.13j     1321.6038   -1668.4622j
  -551.9748   +2049.3252j    364.92923  -1432.4323j
   965.9612    -458.6043j     -3.8820908-1491.8671j
   -71.43536  +2408.607j    -335.1159   -1281.0707j ]
pos=735
snr is 7.46 dB
ber is 2.78E-03
Trial: 2
[  562.1433   -406.7811j   1949.4066   +293.29156j
  1122.534    +592.4838j   1300.679    -372.9453j
  -360.66266  +370.6133j   1353

In [9]:
snr_arr.astype("float32").tofile(f"matplot/snr_vs_angle_poly_011A_{data_rate}bps_{volt}Vrms_{dist}m_7ch_hphydro_rx_vanatta4x2_{datestr}.bin")
ber_arr.astype("float32").tofile(f"matplot/ber_vs_angle_poly_011A_{data_rate}bps_{volt}Vrms_{dist}m_7ch_hphydro_rx_vanatta4x2_{datestr}.bin")